In [1]:
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [2]:
#colunas que escolhi para dados escolares
col_escol = ['CODMEC','ANO','TIPOESC','DEP','Qtde_Docentes','NUM_FUNC','MERENDA','CLFUND',
            'ALFUND','TOTALEJA','TOT_EE','TOTAL_ALUNOS','CL_CRECHE','AL_CRECHE','CL_PRE','AL_PRE','CLEM','ALEM']
col_rendi = ['CODMEC','EFABAND9ANOS8S','EFABAND9ANOS9S','EFABAND7S','EFABAND8S','TOTALEF_ABAND','TOTALEF_APROV',
             'TOTALEF_REPROV','TOTAL_EM_ABAND','TOTAL_EM_REPROV','CONCLUINTES_EM','EPABAND','EPREPROV']
dados_escolares = pd.read_csv('DadosEscolares1996-2015.csv', encoding = 'latin1', usecols = col_escol)
rendimento = pd.read_csv('RendimentoEscolar2000-2015.csv', encoding = 'latin1', usecols = col_rendi)
esc_rmsp = pd.read_csv('ESC2013_RMSP_CEM.csv', encoding = 'latin1')
nota = pd.read_csv('ENEM2015.csv')

C:\Users\giova\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#verificando o tamanho
print ( 'Dados escolares' + str(dados_escolares.shape))
print ( 'Rendimento' + str(rendimento.shape))
print ( 'ESC_RMSP' + str(esc_rmsp.shape))
print( 'Nota' + str(nota.shape))

Dados escolares(104292, 18)
Rendimento(14944, 13)
ESC_RMSP(10937, 136)
Nota(1243, 3)


In [4]:
#Existem escolas que estão no dataframe dados_escolares, rendimento, esc_rmsp que eu não tenho o valor da nota dela
#Preciso retirar escolas que não estão no dataframe nota
dados_escolares = dados_escolares[dados_escolares.CODMEC.isin(nota['CODIGO DA ENTIDADE'])]
rendimento = rendimento[rendimento.CODMEC.isin(nota['CODIGO DA ENTIDADE'])]
esc_rmsp = esc_rmsp[esc_rmsp.CODESC.isin(nota['CODIGO DA ENTIDADE'])]

In [5]:
#retirando as colunas em que mais que 50% dos valores são null
dados_escolares1 = dados_escolares.drop(dados_escolares.columns[dados_escolares.isnull().mean()>0.5], axis = 1)
#apagando as linha com valor NaN
dados_escolares1.dropna(inplace = True, how = 'any')

In [6]:
#como eu abri todas as colunas do esc_rmsp, agora vou retirar as que não desejo usar
col_rmsp = ['ID', 'LONGITUDE', 'LATITUDE', 'CODESCTX','NOMEESC', 'NOMEMUN','CODMUN', 'NOMDIST', 'CODDIST', 'COD_SC',
           'BAIRRO','CEP','END_ESC','NUM_ESC','DDD','TELEFONE','ENS_REG','REG_IN_CR','REG_IN_PE','REG_FU_8A',
           'REG_FU_9A','REG_MED','REG_MED_I','REG_MED_N','REG_MED_P','ENS_ESP','ESP_IN_CR','ESP_IN_PE',
           'ESP_FU_8A','ESP_FU_9A','ESP_MED','ESP_MED_I','ESP_MED_N','ESP_MED_P','ESP_EJA_F','ESP_EJA_M','EJA',
           'EJA_FUND','EJA_MED','EJA_PROJ']
esc_rmsp1 = esc_rmsp.drop(col_rmsp, axis = 1)

In [7]:
esc_rmsp1.drop(esc_rmsp1.columns[esc_rmsp1.isnull().mean() > 0.3], axis=1, inplace = True)
esc_rmsp1.dropna(how='any', axis = 0, inplace = True)

In [8]:
print('Dados escolares', dados_escolares1.isnull().sum().sum())
print('Rendimento', rendimento.isnull().sum().sum())
print('ESC_RMSP1', esc_rmsp1.isnull().sum().sum())

Dados escolares 0
Rendimento 0
ESC_RMSP1 0


In [9]:
#primeiro teste vai ser o mais simples e fácil, usar o maior dataset que tenho
teste1 = pd.merge(dados_escolares1, nota, how = 'inner', left_on = 'CODMEC', right_on = 'CODIGO DA ENTIDADE')
teste1.drop(['ANO', 'TIPOESC', 'CODIGO DA ENTIDADE', 'NUMERO DE PARTICIPANTES'], axis = 1, inplace = True)
dep = {'Privada': 'privada', 'Estadual':'estadual', 'PRIVADA':'privada', 'ESTADUAL':'estadual',
      'privada':'privada', 'PARTICULAR':'privada','Municipal':'municipal','MUNICIPAL':'municipal',
      'Federal':'federal', 'FEDERAL':'federal'}
teste1.DEP = teste1.DEP.map(dep)
teste1 = pd.get_dummies(teste1, 'DEP')

In [10]:
target = teste1.Classificacao
data = teste1.drop('Classificacao', axis =1)

In [11]:
data.CODMEC  = data.CODMEC.astype('float64')
data_TOTALEJA = data.TOTALEJA.astype('float64')
data.TOTAL_ALUNOS = data.TOTAL_ALUNOS.astype('float64')
data.DEP_estadual = data.DEP_estadual.astype('float64')
data.DEP_federal = data.DEP_federal.astype('float64')
data.DEP_municipal = data.DEP_municipal.astype('float64')
data.DEP_privada = data.DEP_privada.astype("float64")
data.dtypes

CODMEC           float64
CL_CRECHE        float64
CL_PRE           float64
AL_CRECHE        float64
AL_PRE           float64
CLEM             float64
ALEM             float64
CLFUND           float64
ALFUND           float64
TOTALEJA         float64
TOTAL_ALUNOS     float64
DEP_estadual     float64
DEP_federal      float64
DEP_municipal    float64
DEP_privada      float64
dtype: object

In [12]:
target.value_counts()

2    7249
3    1355
1    1130
4     656
0       3
Name: Classificacao, dtype: int64

In [14]:
def modelo_svm_kf(data, target, gamma, C, k):
    kf = KFold(n_splits = k)
    scores = []
    for g in gamma:
            for c in C:
                mean_kf = []
                for train_i, test_i in kf.split(data):
                    #separação dos dados em treino e teste
                    data_train, data_test = data.iloc[train_i], data.iloc[test_i]
                    target_train, target_test = target[train_i], target[test_i]
                    
                    #normalização dos dados
                    scaler = StandardScaler()
                    scaler.fit(data_train)
                    data_train = scaler.transform(data_train)
                    data_test = scaler.transform(data_test)
                    
                    #pca nos dados
                    pca = PCA(0.95)
                    pca.fit(data_train)
                    data_train = pca.transform(data_train)
                    data_test = pca.transform(data_test)
                    
                    #rodando o svm
                    svc = SVC(kernel = 'rbf', C=c, gamma = g)
                    svc.fit(data_train,target_train)
                    mean_kf.append(svc.score(data_test,target_test))
                                   
                #salvando a média do KFold para g e c dados
                scores.append(((sum(mean_kf)/len(mean_kf)), g, c))
                print('Iteração com gamma %f e c %f concluída' %(g, c))
    return scores

In [15]:
modelo_svm_kf(data, target, [15], [15], 5)

Iteração com gamma 15.000000 e c 15.000000 concluída


[(0.6400440076089741, 15, 15)]

In [23]:
gamma = np.linspace(0.1,20,15)
c = gamma
svm1 = modelo_svm_kf(data, target, gamma, c,5)

Iteração com gamma 0.100000 e c 0.100000 concluída
Iteração com gamma 0.100000 e c 1.521429 concluída
Iteração com gamma 0.100000 e c 2.942857 concluída
Iteração com gamma 0.100000 e c 4.364286 concluída
Iteração com gamma 0.100000 e c 5.785714 concluída
Iteração com gamma 0.100000 e c 7.207143 concluída
Iteração com gamma 0.100000 e c 8.628571 concluída
Iteração com gamma 0.100000 e c 10.050000 concluída
Iteração com gamma 0.100000 e c 11.471429 concluída
Iteração com gamma 0.100000 e c 12.892857 concluída
Iteração com gamma 0.100000 e c 14.314286 concluída
Iteração com gamma 0.100000 e c 15.735714 concluída
Iteração com gamma 0.100000 e c 17.157143 concluída
Iteração com gamma 0.100000 e c 18.578571 concluída
Iteração com gamma 0.100000 e c 20.000000 concluída
Iteração com gamma 1.521429 e c 0.100000 concluída
Iteração com gamma 1.521429 e c 1.521429 concluída
Iteração com gamma 1.521429 e c 2.942857 concluída
Iteração com gamma 1.521429 e c 4.364286 concluída
Iteração com gamma 1.52

Iteração com gamma 14.314286 e c 12.892857 concluída
Iteração com gamma 14.314286 e c 14.314286 concluída
Iteração com gamma 14.314286 e c 15.735714 concluída
Iteração com gamma 14.314286 e c 17.157143 concluída
Iteração com gamma 14.314286 e c 18.578571 concluída
Iteração com gamma 14.314286 e c 20.000000 concluída
Iteração com gamma 15.735714 e c 0.100000 concluída
Iteração com gamma 15.735714 e c 1.521429 concluída
Iteração com gamma 15.735714 e c 2.942857 concluída
Iteração com gamma 15.735714 e c 4.364286 concluída
Iteração com gamma 15.735714 e c 5.785714 concluída
Iteração com gamma 15.735714 e c 7.207143 concluída
Iteração com gamma 15.735714 e c 8.628571 concluída
Iteração com gamma 15.735714 e c 10.050000 concluída
Iteração com gamma 15.735714 e c 11.471429 concluída
Iteração com gamma 15.735714 e c 12.892857 concluída
Iteração com gamma 15.735714 e c 14.314286 concluída
Iteração com gamma 15.735714 e c 15.735714 concluída
Iteração com gamma 15.735714 e c 17.157143 concluída


In [24]:
svm1.sort()
svm1[-5:]

[(0.6972936663023285, 11.47142857142857, 0.1),
 (0.6973898201039683, 14.314285714285711, 0.1),
 (0.6973898663985285, 10.049999999999999, 0.1),
 (0.6974859276110479, 7.2071428571428555, 0.1),
 (0.6985438971964476, 0.1, 0.1)]